In [1]:
using CSV, DataFrames

In [2]:
primAuthor = "Stoleriu"
fileOwnPapers = "LS_savedrecs_mine.txt"
fileCitations = "LS_savedrecs_848.txt"

"LS_savedrecs_848.txt"

In [3]:
#root = dirname(@__FILE__)
root = dirname("/mnt/e/Stoleriu/Doc/CV/auto/")
inputAIS = joinpath(root, "AI_factor_full_1997-2021.txt")
inputISI = joinpath(root, "IF_factor_full_1997-2021.txt")

"/mnt/e/Stoleriu/Doc/CV/auto/IF_factor_full_1997-2021.txt"

Pentru calculele pe 5 ani:

In [4]:
ANini = 2017
ANfin = 2021

2021

In [5]:
AIS = CSV.read(inputAIS, DataFrame; header=true)
AIS[8157, [1, 2, 3, 14, 25, 26]]

ArgumentError: ArgumentError: "/mnt/e/Stoleriu/Doc/CV/auto/AI_factor_full_1997-2021.txt" is not a valid file or doesn't exist

In [6]:
ISI = CSV.read(inputISI, DataFrame; header=true)
ISI[8157, [1, 2, 3, 14, 25, 26]]

ArgumentError: ArgumentError: "/mnt/e/Stoleriu/Doc/CV/auto/IF_factor_full_1997-2021.txt" is not a valid file or doesn't exist

In [7]:
#root = dirname(@__FILE__)
root = dirname("/mnt/e/Stoleriu/Doc/CV/auto/")
inputMINE = joinpath(root, fileOwnPapers)
inputCIT = joinpath(root, fileCitations)

"/mnt/e/Stoleriu/Doc/CV/auto/LS_savedrecs_848.txt"

In [8]:
MINEraw = CSV.read(inputMINE, DataFrame; header=true)
eltype.(eachcol(MINEraw));

In [9]:
MINE = MINEraw[:, [2, 9, 24, 30, 32, 42, 45, 46, 47, 52, 53, 54, 55]];
eltype.(eachcol(MINE));

In [10]:
MINE[1, :PY] = 2021   # ADV MAT no year!!!!
eltype.(eachcol(MINE));

ArgumentError: ArgumentError: Cannot assign to non-existent column: PY

In [11]:
#root = dirname("/mnt/d/Stoleriu/")
#outputMINE = joinpath(root, "MINE_test.csv")
#CSV.write(outputMINE, MINE)

Pregatim MINE cu noi coloane

In [12]:
rename!(MINE,:"J9" => :"Journal")
insertcols!(MINE, :NA => 0);
insertcols!(MINE, :NEFF => 0.0);
insertcols!(MINE, :PRIM => 0);
insertcols!(MINE, :AI => 0.0);
insertcols!(MINE, :AINEFF => 0.0);
insertcols!(MINE, :IF => 0.0);
insertcols!(MINE, :IFNA => 0.0);
insertcols!(MINE, :TIMESC => 0);
insertcols!(MINE, :CT => 0.0);
### 5 ani
insertcols!(MINE, :PRIM5 => 0);
insertcols!(MINE, :AINEFF5 => 0.0);
insertcols!(MINE, :IFNA5 => 0.0);
insertcols!(MINE, :TIMESC5 => 0);
insertcols!(MINE, :CT5 => 0.0);
### UAIC
insertcols!(MINE, :UAIC1 => 0.0);
insertcols!(MINE, :UAIC9 => 0.0);

ArgumentError: ArgumentError: Tried renaming :J9 to :Journal, when :J9 does not exist in the Index.

Gasim numarul de autori si prim autor

In [13]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    MINE[row_index, :NA] = (1+count(";", lucrare[:AU]))

    if MINE[row_index, :NA] <= 5
        MINE[row_index, :NEFF] = MINE[row_index, :NA]
    elseif MINE[row_index, :NA] <= 15
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 5.0) / 2.0
    elseif MINE[row_index, :NA] <= 75
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 15.0) / 3.0
    else
        MINE[row_index, :NEFF] = (MINE[row_index, :NA] + 45.0) / 4.0
    end

    if findfirst(primAuthor, MINE[row_index, :AU])[1] == 1
        MINE[row_index, :PRIM] = 1
        if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini)
            MINE[row_index, :PRIM5] = 1
        end
    elseif occursin(primAuthor, lucrare[:RP])
        MINE[row_index, :PRIM] = 1
        if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini)
            MINE[row_index, :PRIM5] = 1
        end
    end
    #println(row_index, MINE[row_index, [:NA]])
    #println(row_index, MINE[row_index, [:PRIM]])
end

ArgumentError: ArgumentError: column name :AU not found in the data frame; existing most similar names are: :$AU$

In [14]:
#MINE[:, [6, 7, 8, 12, 14, 15, 16, 17]]
MINE.PY = Int64.(MINE.PY)
MINE[ismissing.(MINE[!, :PY]), [1, 6, 7, 8, 12, 14, 15, 16, 17]]

ArgumentError: ArgumentError: column name :PY not found in the data frame; existing most similar names are: :$PY$

In [15]:
#root = dirname("/mnt/d/Stoleriu/")
#outputMINE = joinpath(root, "MINE_test.csv")
#CSV.write(outputMINE, MINE)

In [16]:
AISsubsect = filter(r -> any(occursin.(Vector(MINE[1, [:Journal]]), r.Journal)), AIS)
#subsect2 = filter(r -> (["ADV MATER"][1] .== r.Journal[1]), AIS)
size(AISsubsect, 1)
#["ADV MATER"][1] .== subsect[1, [:Journal]][1]

UndefVarError: UndefVarError: AIS not defined

In [17]:
for (row_index, lucrare) in enumerate(eachrow(MINE))
    #lucrare = MINE[1, :]
    if(ismissing(lucrare[:Journal]))
        continue
    end
    for AISrow in eachrow(AIS)
        if( lucrare[:Journal] == AISrow[:Journal] )
            anString = string(lucrare[:PY])
            #println(AISrow[anString])
            MINE[row_index, :AI] = AISrow[anString]
            MINE[row_index, :AINEFF] = MINE[row_index, :AI] / MINE[row_index, :NEFF]
            if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini)
                MINE[row_index, :AINEFF5] = MINE[row_index, :AINEFF]
            end
            
        end
    end
    for ISIrow in eachrow(ISI)
        if( lucrare[:Journal] == ISIrow[:Journal] )
            anString = string(lucrare[:PY])
            #println(ISIrow[anString])
            MINE[row_index, :IF] = ISIrow[anString]
            MINE[row_index, :IFNA] = ISIrow[anString] / MINE[row_index, :NA]
            if (MINE[row_index, :PY] <= ANfin) && (MINE[row_index, :PY] >= ANini)
                MINE[row_index, :IFNA5] = MINE[row_index, :IFNA]
                MINE[row_index, :UAIC1] = (25 + 60*MINE[row_index, :IF]) / MINE[row_index, :NA]
            end
        end
    end
end

ArgumentError: ArgumentError: column name :Journal not found in the data frame

In [18]:
#root = dirname("/mnt/d/Stoleriu/")
#outputMINE = joinpath(root, "MINE_test.csv")
#CSV.write(outputMINE, MINE)

In [19]:
I = sum(MINE[!, :AINEFF])

ArgumentError: ArgumentError: column name :AINEFF not found in the data frame

In [20]:
P = sum(MINE[!, :PRIM] .* MINE[!, :AI])

ArgumentError: ArgumentError: column name :PRIM not found in the data frame

In [21]:
I5 = sum(MINE[!, :AINEFF5])

ArgumentError: ArgumentError: column name :AINEFF5 not found in the data frame

In [22]:
P5 = sum(MINE[!, :PRIM5] .* MINE[!, :AI])

ArgumentError: ArgumentError: column name :PRIM5 not found in the data frame

In [23]:
UAIC_1_1 = sum(MINE[!, :UAIC1])

ArgumentError: ArgumentError: column name :UAIC1 not found in the data frame

############################
Citations
############################

In [24]:
MINECITEraw = CSV.read(inputCIT, DataFrame; header=true);
# autocitari
#for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITEraw))
#    if( occursin(primAuthor, lucrareCIT.AU) )
#        println(lucrareCIT.AU)
#    end
#end
eltype.(eachcol(MINECITEraw));

ArgumentError: ArgumentError: "/mnt/e/Stoleriu/Doc/CV/auto/LS_savedrecs_848.txt" is not a valid file or doesn't exist

In [25]:
# eliminare autocitari
MINECITEraw = filter(row -> !(occursin(primAuthor, row.AU)),  MINECITEraw);
for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITEraw))
    if( occursin(primAuthor, lucrareCIT.AU) )
        println(lucrareCIT.AU)
    end
end

UndefVarError: UndefVarError: MINECITEraw not defined

In [26]:
MINECITE = MINECITEraw[:, [2, 9, 30, 42, 45, 46, 47, 52, 53, 54, 55]];
eltype.(eachcol(MINECITE));
rename!(MINECITE,:"J9" => :"Journal");

UndefVarError: UndefVarError: MINECITEraw not defined

In [27]:
#CITES = fill!(Vector{Vector{Int64}}(undef, size(MINE)[1]), [0]);
CITES = Vector{Int64}[];
for i in 1:size(MINE)[1]
    push!(CITES, [0])
end
CITES_IFs = Vector{Float64}[];
for i in 1:size(MINE)[1]
    push!(CITES_IFs, [0.0])
end

In [28]:
CITES5 = Vector{Int64}[];
for i in 1:size(MINE)[1]
    push!(CITES5, [0])
end
CITES5_IFs = Vector{Float64}[];
for i in 1:size(MINE)[1]
    push!(CITES5_IFs, [0.0])
end

In [29]:
for (row_indexMINE, lucrareMINE) in enumerate(eachrow(MINE))
    calcul_cit_UAIC = 0.0
    if ( ismissing(lucrareMINE[:DI]) )
        #println("Skip paper $row_indexMINE noDOI")
        continue
    end
    #println("Analyzing paper ", lucrareMINE[:DI])
    for (row_indexCIT, lucrareCIT) in enumerate(eachrow(MINECITE))
        if ( occursin(lucrareMINE[:DI], lucrareCIT[:CR]) )
            #println("Paper $row_indexMINE cited by $row_indexCIT")
            ISI_value = 0.0
            push!(CITES[row_indexMINE], row_indexCIT)
            push!(CITES_IFs[row_indexMINE], ISI_value)
            if (( !ismissing(lucrareCIT[:PY]) ) && ( !ismissing(lucrareCIT[:Journal]) ))
                for ISIrow in eachrow(ISI)
                    if( lucrareCIT[:Journal] == ISIrow[:Journal] )
                        an_publicare_citare = lucrareCIT[:PY]
                        if (an_publicare_citare > ANfin)
                            an_publicare_citare = ANfin
                        end
                        ISI_value = ISIrow[string(an_publicare_citare)]
                        push!(CITES_IFs[row_indexMINE], ISI_value)
                        #CITES_IFs[row_indexMINE, size(CITES_IFs[row_indexMINE])] = ISI_value
                        break
                    end
                end
                if (lucrareCIT[:PY] <= ANfin) && (lucrareCIT[:PY] >= ANini)
                    push!(CITES5[row_indexMINE], row_indexCIT)
                    push!(CITES5_IFs[row_indexMINE], ISI_value)
                    calcul_cit_UAIC = calcul_cit_UAIC + (10 + 20 * ISI_value) / lucrareMINE[:NA]
                end
            end
        end
    end
    MINE[row_indexMINE, :TIMESC] = size(CITES[row_indexMINE])[1]-1
    MINE[row_indexMINE, :CT] = MINE[row_indexMINE, :TIMESC] / MINE[row_indexMINE, :NEFF]
    MINE[row_indexMINE, :TIMESC5] = size(CITES5[row_indexMINE])[1]-1
    MINE[row_indexMINE, :CT5] = MINE[row_indexMINE, :TIMESC5] / MINE[row_indexMINE, :NEFF]
    MINE[row_indexMINE, :UAIC9] = calcul_cit_UAIC

end;

ArgumentError: ArgumentError: column name :DI not found in the data frame; existing most similar names are: :$DI$

In [30]:
CITES5[2]

1-element Vector{Int64}:
 0

In [31]:
size(CITES5_IFs[2])

(1,)

In [32]:
C = sum(MINE[!, :CT])

ArgumentError: ArgumentError: column name :CT not found in the data frame

In [33]:
C5 = sum(MINE[!, :CT5])

ArgumentError: ArgumentError: column name :CT5 not found in the data frame

In [34]:
UAIC_1_9 = sum(MINE[!, :UAIC9])

ArgumentError: ArgumentError: column name :UAIC9 not found in the data frame

In [35]:
root = dirname("/mnt/e/Stoleriu/Doc/CV/auto/")
outputMINEcsv = joinpath(root, "LS_MINE_test.csv")
CSV.write(outputMINEcsv, MINE)

"/mnt/e/Stoleriu/Doc/CV/auto/LS_MINE_test.csv"

In [36]:
ENV["COLUMNS"]=500
MINE[3, filter(x -> !(x in ["CR","RP"]), names(MINE))]

,$AU$,$TI$,$RP$,$CR$
,String,String,String,String
3,"$Stoleriu, L; Enachescu, C$",$ELASTIC MODEL FOR SPIN CROSSOVER NANOPARTICLES IN MATRICES$,"$Enachescu, C (corresponding author), Alexandru Joan Cuza Univ Iasi, Fac Phys, Bd Carol I 11, Iasi, Romania.$","$Atitoaie A, 2012, J MAGN MAGN MATER, V324, P1596, DOI 10.1016/j.jmmm.2011.12.011; Boldog I, 2008, ANGEW CHEM INT EDIT, V47, P6433, DOI 10.1002/anie.200801673; Boukheddaden K, 2000, PHYS REV B, V62, P14796, DOI 10.1103/PhysRevB.62.14796; Delgado T, 2018, PHYS CHEM CHEM PHYS, V20, P12493, DOI 10.1039/c8cp00775f; Enachescu C, 2016, PHYS CHEM CHEM PHYS, V18, P20591, DOI 10.1039/c6cp02806c; Enachescu C, 2009, PHYS REV LETT, V102, DOI 10.1103/PhysRevLett.102.257204; Gaina R, 2017, P ROMANIAN ACAD A, V18, P215; GUTLICH P, 1994, ANGEW CHEM INT EDIT, V33, P2024, DOI 10.1002/anie.199420241; Kawamoto T, 2005, CHEM COMMUN, P3933, DOI 10.1039/b506643c; Nguyen LL, 2015, NEW J CHEM, V39, P1603, DOI 10.1039/c4nj01257g; Martinez V, 2010, CHEM MATER, V22, P4271, DOI 10.1021/cm101022u; Muraoka A., 2011, PHYS REV B, V5, P84; Oubouchou H, 2013, PHYS REV B, V87, DOI 10.1103/PhysRevB.87.104104; Raza Y, 2011, CHEM COMMUN, V47, P11501, DOI 10.1039/c1cc14463d; Stoleriu L, 2011, PHYS REV B, V84, DOI 10.1103/PhysRevB.84.134102; Tissot A, 2012, J MATER CHEM, V22, P20451, DOI 10.1039/c2jm33865c; Varret F, 2003, CR CHIM, V6, P385, DOI 10.1016/S1631-0748(03)00048-1$"


In [37]:
using Printf
outputMINEtxt = joinpath(root, "LS_UAIC_test.txt")
open(outputMINEtxt, "w") do file
    for (row_indexMINE, lucrareMINE) in enumerate(eachrow(MINE))
        if ( !ismissing(lucrareMINE.BP) )
            write(file, "[$row_indexMINE] \t $(lucrareMINE.AU); $(lucrareMINE.TI); $(lucrareMINE.Journal), vol.$(lucrareMINE.VL), pp.$(lucrareMINE.BP)-$(lucrareMINE.EP) ($(lucrareMINE.PY)), DOI:$(lucrareMINE.DI) \n")
        else
            write(file, "[$row_indexMINE] \t $(lucrareMINE.AU); $(lucrareMINE.TI); $(lucrareMINE.Journal), vol.$(lucrareMINE.VL), art.no.$(lucrareMINE.AR) ($(lucrareMINE.PY)), DOI:$(lucrareMINE.DI) \n")
        end
        contor = 1
        for (idx_citare, citare) in enumerate(CITES5[row_indexMINE])
            if (citare == 0)
                continue
            end
            write(file, "\t\t[$row_indexMINE.$contor] $(MINECITE[citare,:].AU); $(MINECITE[citare,:].TI); $(MINECITE[citare,:].Journal), vol.$(MINECITE[citare,:].VL) ($(MINECITE[citare,:].PY)), DOI:$(MINECITE[citare,:].DI) \n")
            write(file, "\t\t\tIF citare: $(CITES5_IFs[row_indexMINE][idx_citare])\n")
            contor = contor + 1
        end
        write(file, "nr.autori: $(lucrareMINE.NA) \t\t IF: $(lucrareMINE.IF) \t\t UAIC I.1(ISI): $(@sprintf("%8.4f", lucrareMINE.UAIC1)) \t\t nr.citari: $(lucrareMINE.TIMESC5) \t\t UAIC I.9(Citari): $(@sprintf("%9.4f", lucrareMINE.UAIC9)) \n")
        write(file, '-'^400, '\n')
    end
    write(file, '-'^400, '\n')
    write(file, '-'^400, '\n')
    write(file, "Total criteriul I.1(UAIC) = $(@sprintf("%9.4f", UAIC_1_1))\t\t\tTotal criteriul I.9(UAIC) = $(@sprintf("%9.4f", UAIC_1_9))")
end

ArgumentError: ArgumentError: column name :BP not found in the data frame; existing most similar names are: :$BP$